In [34]:
import pandas as pd
import numpy as np

In [2]:
# считаем описание данных
df = pd.read_csv('02_otp_desc.csv', sep=';').loc[:,'ПОЛЕ':'ОПИСАНИЕ']

In [3]:
df

,ПОЛЕ,ОПИСАНИЕ
0,AGREEMENT_RK,уникальный идентификатор объекта в выборке
1,TARGET,целевая переменная:отклик на маркетинговую кам...
2,AGE,возраст клиента
3,SOCSTATUS_WORK_FL,социальный статус клиента относительно работы ...
4,SOCSTATUS_PENS_FL,социальный статус клиента относительно пенсии ...
5,GENDER,пол клиента
6,CHILD_TOTAL,количество детей клиента
7,DEPENDANTS,количество иждивенцев клиента
8,EDUCATION,образование
9,MARITAL_STATUS,семейное положение


In [4]:
# считаем все-все данные
train = pd.read_csv('02_otp_train.csv', sep=';')
train.shape

(15223, 52)

In [5]:
test = pd.read_csv('02_otp_test.csv', sep=';')
test.shape

(14910, 51)

In [6]:
# отделим таргет от TRAIN-а
y = train['TARGET']
train = train.drop('TARGET', axis=1)

In [7]:
# соеденим train и test
train['is_test'] = False
test['is_test'] = True
all_data = pd.concat([train, test])
all_data.shape

(30133, 52)

In [8]:
# необходимо удалить индекс
# у нас есть 0-строка в train и 0-строка в test (и так далее 1,2,3,4)
# если мы просто объеденим - у нас сохранятся индексы
# мы же просто уберем индекс и автоматически создастся новый индекс, сквозной и уникальный
all_data = all_data.reset_index(drop=True)

In [9]:
# посмотрим типы данных и их заполняемость
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30133 entries, 0 to 30132
Data columns (total 52 columns):
AGREEMENT_RK                  30133 non-null int64
AGE                           30133 non-null int64
SOCSTATUS_WORK_FL             30133 non-null int64
SOCSTATUS_PENS_FL             30133 non-null int64
GENDER                        30133 non-null int64
CHILD_TOTAL                   30133 non-null int64
DEPENDANTS                    30133 non-null int64
EDUCATION                     30133 non-null object
MARITAL_STATUS                30133 non-null object
GEN_INDUSTRY                  27420 non-null object
GEN_TITLE                     27420 non-null object
ORG_TP_STATE                  27420 non-null object
ORG_TP_FCAPITAL               27425 non-null object
JOB_DIR                       27420 non-null object
FAMILY_INCOME                 30133 non-null object
PERSONAL_INCOME               30133 non-null object
REG_ADDRESS_PROVINCE          30133 non-null object
FACT_ADDRESS_P

In [10]:
# Видим, что часть данных - object, скорее всего стоки.
# Давайте выведем эти значения для каждого столбца

for i in all_data.columns: # перебираем все столбцы
    if str(all_data[i].dtype) == 'object': # если тип столбца - object
        print(i) # выводим название столбца
        print(set(all_data[i])) # выводим все его значения (но делаем set - чтоб значения не повторялись)
        print() # выводим пустую строку

EDUCATION
{'Неоконченное высшее', 'Среднее специальное', 'Неполное среднее', 'Высшее', 'Среднее', 'Ученая степень', 'Два и более высших образования'}

MARITAL_STATUS
{'Вдовец/Вдова', 'Не состоял в браке', 'Гражданский брак', 'Состою в браке', 'Разведен(а)'}

GEN_INDUSTRY
{nan, 'Логистика', 'Здравоохранение', 'Страхование', 'Коммунальное хоз-во/Дорожные службы', 'Сельское хозяйство', 'Юридические услуги/нотариальные услуги', 'Недвижимость', 'Туризм', 'Нефтегазовая промышленность', 'Химия/Парфюмерия/Фармацевтика', 'СМИ/Реклама/PR-агенства', 'Транспорт', 'Подбор персонала', 'Энергетика', 'Государственная служба', 'Информационные технологии', 'Образование', 'Наука', 'Банк/Финансы', 'Другие сферы', 'Управляющая компания', 'ЧОП/Детективная д-ть', 'Информационные услуги', 'Ресторанный бизнес/Общественное питание', 'Сборочные производства', 'Строительство', 'Салоны красоты и здоровья', 'Торговля', 'Маркетинг', 'Развлечения/Искусство', 'Металлургия/Промышленность/Машиностроение'}

GEN_TITLE
{na

In [11]:
# можно заметить что некоторые переменные, 
# которые обозначены как строки (например PERSONAL_INCOME) на самом деле числа,
# но по какой-то (ниже увидим) причине были распознаны как строки
all_data['PERSONAL_INCOME'] = all_data['PERSONAL_INCOME'].map(lambda x: x.replace(',', '.')).astype('float')

In [12]:
# причина же что использовалась запятая для разделения не целой части числа
all_data['CREDIT'] = all_data['CREDIT'].map(lambda x: x.replace(',', '.')).astype('float')
all_data['FST_PAYMENT'] = all_data['FST_PAYMENT'].map(lambda x: x.replace(',', '.')).astype('float')
all_data['LOAN_AVG_DLQ_AMT'] = all_data['LOAN_AVG_DLQ_AMT'].map(lambda x: x.replace(',', '.')).astype('float')
all_data['LOAN_MAX_DLQ_AMT'] = all_data['LOAN_MAX_DLQ_AMT'].map(lambda x: x.replace(',', '.')).astype('float')

In [13]:
# запускаем повторно код и смотрим, какие столбцы с какими строковыми значениями остались
for i in all_data.columns:
    if str(all_data[i].dtype) == 'object':
        print(i)
        print(set(all_data[i]))
        print()

EDUCATION
{'Неоконченное высшее', 'Среднее специальное', 'Неполное среднее', 'Высшее', 'Среднее', 'Ученая степень', 'Два и более высших образования'}

MARITAL_STATUS
{'Вдовец/Вдова', 'Не состоял в браке', 'Гражданский брак', 'Состою в браке', 'Разведен(а)'}

GEN_INDUSTRY
{nan, 'Логистика', 'Здравоохранение', 'Страхование', 'Коммунальное хоз-во/Дорожные службы', 'Сельское хозяйство', 'Юридические услуги/нотариальные услуги', 'Недвижимость', 'Туризм', 'Нефтегазовая промышленность', 'Химия/Парфюмерия/Фармацевтика', 'СМИ/Реклама/PR-агенства', 'Транспорт', 'Подбор персонала', 'Энергетика', 'Государственная служба', 'Информационные технологии', 'Образование', 'Наука', 'Банк/Финансы', 'Другие сферы', 'Управляющая компания', 'ЧОП/Детективная д-ть', 'Информационные услуги', 'Ресторанный бизнес/Общественное питание', 'Сборочные производства', 'Строительство', 'Салоны красоты и здоровья', 'Торговля', 'Маркетинг', 'Развлечения/Искусство', 'Металлургия/Промышленность/Машиностроение'}

GEN_TITLE
{na

# Данные готовы, теперь вам предлагается начать решать задачу

In [14]:
# Есть ли пропуски в данных? что с ними сделать?
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30133 entries, 0 to 30132
Data columns (total 52 columns):
AGREEMENT_RK                  30133 non-null int64
AGE                           30133 non-null int64
SOCSTATUS_WORK_FL             30133 non-null int64
SOCSTATUS_PENS_FL             30133 non-null int64
GENDER                        30133 non-null int64
CHILD_TOTAL                   30133 non-null int64
DEPENDANTS                    30133 non-null int64
EDUCATION                     30133 non-null object
MARITAL_STATUS                30133 non-null object
GEN_INDUSTRY                  27420 non-null object
GEN_TITLE                     27420 non-null object
ORG_TP_STATE                  27420 non-null object
ORG_TP_FCAPITAL               27425 non-null object
JOB_DIR                       27420 non-null object
FAMILY_INCOME                 30133 non-null object
PERSONAL_INCOME               30133 non-null float64
REG_ADDRESS_PROVINCE          30133 non-null object
FACT_ADDRESS_

In [15]:
# Удалим все строковые фичи
all_data = all_data.drop(['GEN_INDUSTRY', 'GEN_INDUSTRY', 'GEN_TITLE', 'ORG_TP_STATE', 'ORG_TP_FCAPITAL', 'JOB_DIR', 'PERSONAL_INCOME', 'REG_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'TP_PROVINCE', 'REGION_NM', 'PREVIOUS_CARD_NUM_UTILIZED'], axis=1)

In [16]:
#all_data['PREVIOUS_CARD_NUM_UTILIZED'].replace(to_replace = 'Null', value="0", inplace=True)

In [17]:
all_data.WORK_TIME[all_data.WORK_TIME.isnull()] = all_data.WORK_TIME.mean()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
# Как поступить со строковыми переменными?
# Переведем строковые значения признака FAMILY_INCOME в значения от 0 до 4
#'от 20000 до 50000 руб.', 'до 5000 руб.', 'от 10000 до 20000 руб.', 'от 5000 до 10000 руб.', 'свыше 50000 руб.'

all_data['FAMILY_INCOME'].replace(to_replace = 'до 5000 руб.', value="0", inplace=True)
all_data['FAMILY_INCOME'].replace(to_replace = 'от 5000 до 10000 руб.', value="1", inplace=True)
all_data['FAMILY_INCOME'].replace(to_replace = 'от 10000 до 20000 руб.', value="2", inplace=True)
all_data['FAMILY_INCOME'].replace(to_replace = 'от 20000 до 50000 руб.', value="3", inplace=True)
all_data['FAMILY_INCOME'].replace(to_replace = 'свыше 50000 руб.', value="4", inplace=True)
all_data['FAMILY_INCOME'] = all_data.FAMILY_INCOME.astype('int64')

In [19]:
all_data = pd.get_dummies(all_data, columns=['EDUCATION', 'MARITAL_STATUS'])

In [20]:
# В итоге должны получиться данные без пропусков и только числовые
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30133 entries, 0 to 30132
Data columns (total 50 columns):
AGREEMENT_RK                                30133 non-null int64
AGE                                         30133 non-null int64
SOCSTATUS_WORK_FL                           30133 non-null int64
SOCSTATUS_PENS_FL                           30133 non-null int64
GENDER                                      30133 non-null int64
CHILD_TOTAL                                 30133 non-null int64
DEPENDANTS                                  30133 non-null int64
FAMILY_INCOME                               30133 non-null int64
REG_FACT_FL                                 30133 non-null int64
FACT_POST_FL                                30133 non-null int64
REG_POST_FL                                 30133 non-null int64
REG_FACT_POST_FL                            30133 non-null int64
REG_FACT_POST_TP_FL                         30133 non-null int64
FL_PRESENCE_FL                              30

In [21]:
# Вновь разделите на train и test (не мешая)
train = all_data[~all_data.is_test].drop(['is_test'], axis=1)
test = all_data[all_data.is_test].drop(['is_test'], axis=1)
train.shape

(15223, 49)

In [22]:
# Сделать hold-out sampling на train выборке
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3, random_state=42)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [23]:
X_train.shape
X_test.shape
print(X_train.shape, X_test.shape)

(10656, 49) (4567, 49)


In [24]:
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score, f1_score

# Первая модель - Дерево решений

In [25]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
predict_dtc = dtc.predict(X_test)

print('precision', precision_score(y_test, predict_dtc))
print('recall', recall_score(y_test, predict_dtc))
print('accuracy', accuracy_score(y_test, predict_dtc))
print('f1', f1_score(y_test, predict_dtc))

precision 0.186356073211
recall 0.201801801802
accuracy 0.795927304576
f1 0.193771626298


In [26]:
predict_dtc[:1]

array([0], dtype=int64)

# Вторая модель - Случай лес

In [27]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
predict_rfc = rfc.predict(X_test)

print('precision', precision_score(y_test, predict_rfc))
print('recall', recall_score(y_test, predict_rfc))
print('accuracy', accuracy_score(y_test, predict_rfc))
print('f1', f1_score(y_test, predict_rfc))

precision 0.227272727273
recall 0.00900900900901
accuracy 0.875848478213
f1 0.0173310225303


# Третья модель - Лог.регрессия

In [28]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
log.fit(X_train, y_train)
predict_log = log.predict(X_test)

print('precision', precision_score(y_test, predict_log))
print('recall', recall_score(y_test, predict_log))
print('accuracy', accuracy_score(y_test, predict_log))
print('f1', f1_score(y_test, predict_log))

precision 0.0
recall 0.0
accuracy 0.878476023648
f1 0.0


C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
predict_log

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [30]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
fpr, tpr, thresholds = roc_curve(y_test, predict_log[:,1])
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1])
plt.ylabel('tpr')
plt.xlabel('fpr')
plt.grid(True)
plt.title('ROC curve')
plt.xlim((-0.01, 1.01))
plt.ylim((-0.01, 1.01))

IndexError: too many indices for array

In [ ]:
# Потроить roc-кривые всех моделей на одном графике
# Вывести roc_auc каждой моделе

# Ваш код :-)

In [39]:
from sklearn.cross_validation import cross_val_score
# Сделать k-fold (10 фолдов) кросс-валидацию каждой модели и посчитать средний roc_auc

rfc = RandomForestClassifier()
print('10 Folds',cross_val_score(rfc, X_train, y_train, scoring='roc_auc', cv=10))
print('mean',np.mean(cross_val_score(rfc, X_train, y_train, scoring='roc_auc', cv=10)))

10 Folds [ 0.6220027   0.58927305  0.58344732  0.57376731  0.60343634  0.59732776
  0.60108072  0.55552766  0.57991489  0.53769968]
mean 0.580655815775


In [37]:
# Взять лучшую модель и сделать predict (с вероятностями (!!!)) для test выборки

# Ваш код :-)